# Modeling

I will be using two different modeling techniques and determine which one is more effective than the other. The first model we will use is a random forest model, this model is built upon using the idea of making many decision trees and aggregating these trees together. After there in respective groups, the trees then take a vote and we then get the prediction that is the best from all of them. The second one is a decision tree, which contructs the best tree it can by determining the best predictors for the nodes. 

In [39]:
#Im going to steal my professors datascience libraries off of dropbox
import os
home_path =  os.path.expanduser('~')
os.chdir(home_path + '/Dropbox/cis399_ds1_f17/week_libraries/datascience_1')
!git pull

WindowsError: [Error 3] The system cannot find the path specified: 'C:\\Users\\Snapu/Dropbox/cis399_ds1_f17/week_libraries/datascience_1'

In [40]:
import pandas as pd
import sys

url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSFiUDmWZFBaJDsSzbZmlPNjdlfhwtpz2A5ceAHsHL4_GMw7q1FlGok3WAwDbA2aJmx9lxZOVad8TUs/pub?gid=1411702209&single=true&output=csv'
skate_table = pd.read_csv(url)

pd.__version__  # should see 0.20.3 or higher

#This is the local file of my functions 

sys.path.append(home_path + '\Documents\CIS 399\datascience_1')
from week7 import *
%who function




accuracy	 build_pred	 build_tree_iter	 caser	 closest_centroid	 compute_mean	 compute_prediction	 compute_training	 euclidean_distance	 
f1	 find_best_splitter	 forest_builder	 generate_table	 gig	 gini	 informedness	 initialize_centroids	 k_fold	 
k_means	 oob_testing	 phase_1	 phase_2	 predictor_case	 probabilities	 row_to_vect	 seeder	 tree_predictor	 
vote_taker	 


In [41]:
pd.set_option('display.max_columns', None)
skate_table.head(10)

,Season,Series,City,Country,Year,Month,Day,Distance,Round,Group,Num_Skater,Name,Nationality,Rank_In_Group,Start_Position,Time,Qualification,rank_lap1,time_lap1,rank_lap2,time_lap2,rank_lap3,time_lap3,rank_lap4,time_lap4,rank_lap5,time_lap5,kmeans_time_lap5,filled_time,winning
0,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,153,AN Victor',RUS,1,1,41.311,Q,1,6.94,1,8.70,1,8.37,1,8.53,1,8.78,8.78,41.311,0
1,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,114,SHI Jingnan',CHN,2,3,41.358,Q,2,7.12,2,8.68,2,8.27,2,8.56,2,8.73,8.73,41.358,0
2,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,129,David Mano LOTH',HUN,3,2,43.338,?,3,7.15,3,9.30,3,8.76,3,8.93,3,9.2,9.20,43.338,0
3,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,155,Vladimir GRIGOREV',RUS,1,3,40.984,Q,2,7.03,2,8.85,2,8.24,2,8.37,1,8.5,8.50,40.984,1
4,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,165,J R CELSKI',USA,2,1,40.992,Q,1,6.88,1,8.87,1,8.29,1,8.36,2,8.6,8.60,40.992,1
5,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,103,Nathaniel HENRY',AUS,3,2,42.665,?,3,7.04,3,9.11,3,8.66,3,8.88,3,8.98,8.98,42.665,0
6,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,186,Yonaton Spencer SUBIN',ISR,4,4,47.282,?,4,7.89,4,9.97,4,9.74,4,9.75,4,9.94,9.94,47.282,0
7,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,3,143,LEE Han-Bin',KOR,1,2,41.643,Q,1,7.11,1,9.01,1,8.44,1,8.47,1,8.62,8.62,41.643,0
8,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,3,130,BYKANOV Vladislav',ISR,2,4,41.72,Q,4,7.41,3,9.12,2,8.29,2,8.31,2,8.59,8.59,41.720,0
9,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,3,161,Serhiy LIFYRENKO',UKR,3,3,42.58,?,3,7.19,2,9.16,3,8.54,3,8.73,3,8.96,8.96,42.580,0


## One Hot Encoding

I use pandas to separate all the unique values of each column and turn them into binary columns. This allows me to use the data set to construct binary trees, which will be how we will model the speed skaters.

Here I separate the city, nationality, round, year and the lap times being on average less than 8.2 seconds.


In [42]:
ohe_city =  pd.get_dummies(skate_table['City'], prefix = 'city', dummy_na=True)
skate_table = skate_table.join(ohe_city)

ohe_round =  pd.get_dummies(skate_table['Round'], prefix = 'round', dummy_na=True)
skate_table = skate_table.join(ohe_round)

ohe_year =  pd.get_dummies(skate_table['Year'], prefix = 'year', dummy_na=True)
skate_table = skate_table.join(ohe_year)


The lap timing being on par is so we can detect if there is a certain strategy which causes these times, or if it raw speed. I also generate the target column to be binary so we can use it in our trees. 

In [43]:
skate_table['time_lap1_par'] = skate_table.apply(lambda row: 1 if row.time_lap1 < 8.2 else 0, axis = 1)
skate_table['time_lap2_par'] = skate_table.apply(lambda row: 1 if row.time_lap2 < 8.2 else 0, axis = 1)
skate_table['time_lap3_par'] = skate_table.apply(lambda row: 1 if row.time_lap3 < 8.2 else 0, axis = 1)
skate_table['time_lap4_par'] = skate_table.apply(lambda row: 1 if row.time_lap4 < 8.2 else 0, axis = 1)
skate_table['time_lap5_par'] = skate_table.apply(lambda row: 1 if row.kmeans_time_lap5 < 8.2 else 0, axis = 1)
skate_table['winning'] = skate_table.apply(lambda row: 1 if row.filled_time < 41.0 else 0, axis = 1)
skate_table.head()

,Season,Series,City,Country,Year,Month,Day,Distance,Round,Group,Num_Skater,Name,Nationality,Rank_In_Group,Start_Position,Time,Qualification,rank_lap1,time_lap1,rank_lap2,time_lap2,rank_lap3,time_lap3,rank_lap4,time_lap4,rank_lap5,time_lap5,kmeans_time_lap5,filled_time,winning,city_Calgary,city_Debrecen,city_Dordrecht,city_Dresden,city_Erzurum,city_Gangneung,city_Kolomna,city_Malmo,city_Montreal,city_Moscow,city_Nagoya,city_SaltLakeCity,city_Seoul,city_Shanghai,city_Sochi,city_Torino,city_Toronto,city_nan,round_FinalA,round_FinalB,round_Heats,round_PrePreliminaries,round_Preliminaries,round_Quarterfinals,round_RankingRaces,round_RepHeats,round_RepQuarterfinals,round_RepSemifinals,round_Semifinals,round_nan,year_2012.0,year_2013.0,year_2014.0,year_2015.0,year_2016.0,year_nan,time_lap1_par,time_lap2_par,time_lap3_par,time_lap4_par,time_lap5_par
0,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,153,AN Victor',RUS,1,1,41.311,Q,1,6.94,1,8.70,1,8.37,1,8.53,1,8.78,8.78,41.311,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,114,SHI Jingnan',CHN,2,3,41.358,Q,2,7.12,2,8.68,2,8.27,2,8.56,2,8.73,8.73,41.358,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,129,David Mano LOTH',HUN,3,2,43.338,?,3,7.15,3,9.30,3,8.76,3,8.93,3,9.2,9.20,43.338,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,155,Vladimir GRIGOREV',RUS,1,3,40.984,Q,2,7.03,2,8.85,2,8.24,2,8.37,1,8.5,8.50,40.984,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,165,J R CELSKI',USA,2,1,40.992,Q,1,6.88,1,8.87,1,8.29,1,8.36,2,8.6,8.60,40.992,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0


In [44]:
skate_table['winning'].value_counts()

0    4468
1     333
Name: winning, dtype: int64

### Splitters

These splitter columns help us determine which of these categories is going to lend itself to better predicting times under 41 seconds. These categories are what are the possible nodes of our trees, so I think this deserve some paying attention to.

I chose to do variations of City, Round, Year, and Lap Par. 

The city was for to account for the location playing a factor. For example, the cities that have held the Olympics are going to have a higher probability of having skating times under 41. 

Rounds are going to play a factor in the strategy and style of the racer. Rounds also play a factor in fatigue as the event is held in one day

In 2015 there was a rule preventing players from doing "toe-starts". This may have an impact on the frequency of times under 41

Lap par is going to predict and hopefully model strategies of the skaters. 

I’m going to choose to use the City, Year, Round, and Lap Par as those give the best models. If we look we can see that the best tree of depth 1 was the same once we switched from round to city. I think this is because there is some splitter in the round column that is the best for all of them, but regardless the decision tree was consistently underperforming.


In [45]:
skate_table.columns.values

array(['Season', 'Series', 'City', 'Country', 'Year', 'Month', 'Day',
       'Distance', 'Round', 'Group', 'Num_Skater', 'Name', 'Nationality',
       'Rank_In_Group', 'Start_Position', 'Time', 'Qualification',
       'rank_lap1', 'time_lap1', 'rank_lap2', 'time_lap2', 'rank_lap3',
       'time_lap3', 'rank_lap4', 'time_lap4', 'rank_lap5', 'time_lap5',
       'kmeans_time_lap5', 'filled_time', 'winning', 'city_Calgary',
       'city_Debrecen', 'city_Dordrecht', 'city_Dresden', 'city_Erzurum',
       'city_Gangneung', 'city_Kolomna', 'city_Malmo', 'city_Montreal',
       'city_Moscow', 'city_Nagoya', 'city_SaltLakeCity', 'city_Seoul',
       'city_Shanghai', 'city_Sochi', 'city_Torino', 'city_Toronto',
       'city_nan', 'round_FinalA', 'round_FinalB', 'round_Heats',
       'round_PrePreliminaries', 'round_Preliminaries',
       'round_Quarterfinals', 'round_RankingRaces', 'round_RepHeats',
       'round_RepQuarterfinals', 'round_RepSemifinals',
       'round_Semifinals', 'round_nan', '

In [46]:
splitter_columns = [
       'city_Calgary',
       'city_Debrecen', 'city_Dordrecht', 'city_Dresden', 'city_Erzurum',
       'city_Gangneung', 'city_Kolomna', 'city_Malmo', 'city_Montreal',
       'city_Moscow', 'city_Nagoya', 'city_SaltLakeCity', 'city_Seoul',
       'city_Shanghai', 'city_Sochi', 'city_Torino', 'city_Toronto',
       'city_nan', 'round_FinalA', 'round_FinalB', 'round_Heats',
       'round_PrePreliminaries', 'round_Preliminaries',
       'round_Quarterfinals', 'round_RankingRaces', 'round_RepHeats',
       'round_RepQuarterfinals', 'round_RepSemifinals',
       'round_Semifinals', 'round_nan', 'year_2012.0', 'year_2013.0',
       'year_2014.0', 'year_2015.0', 'year_2016.0', 'year_nan',
       'time_lap1_par', 'time_lap2_par', 'time_lap3_par', 'time_lap4_par',
       'time_lap5_par',]

## Random Forests

I will explore the different sizes of the forest and determine which one is the best by I the accuracy, f1, and informedness of each model. I chose these statistics to determine the goodness of our model. The higher the values the better the model is at predicting the condition. This function here is one that takes in a table, possible columns to split upon, the item we are trying to predict, and some hyperparameters.

These hyperparameters help us tweak what is going in this black box. I chose to have different number of trees in the random forest. This changes the size of the voting members, this will change which models are returned


In [47]:
forest1 = forest_builder(skate_table, splitter_columns, 'winning', hypers={'total-trees':5})
len(forest1)

5

In [48]:
skate_table['forest_1'] = skate_table.apply(lambda row: vote_taker(row, forest1), axis=1)
skate_table['forest_1_type'] = skate_table.apply(lambda row: predictor_case(row, pred='forest_1', target='winning'), axis=1)
forest1_types = skate_table['forest_1_type'].value_counts()
print((accuracy(forest1_types), f1(forest1_types), informedness(forest1_types)))


(0.9306394501145595, 0.0, 0.0)


In [49]:
forest2 = forest_builder(skate_table, splitter_columns, 'winning', hypers={'total-trees':11})
len(forest2)

11

In [50]:
skate_table['forest_2'] = skate_table.apply(lambda row: vote_taker(row, forest2), axis=1)
skate_table['forest_2_type'] = skate_table.apply(lambda row: predictor_case(row, pred='forest_2', target='winning'), axis=1)
forest2_types = skate_table['forest_2_type'].value_counts()
print((accuracy(forest2_types), f1(forest2_types), informedness(forest2_types)))


(0.9370964382420329, 0.27403846153846156, 0.16535201271100997)


In [51]:
forest3 = forest_builder(skate_table, splitter_columns, 'winning', hypers={'total-trees':17})
len(forest3)

17

In [52]:
skate_table['forest_3'] = skate_table.apply(lambda row: vote_taker(row, forest3), axis=1)
skate_table['forest_3_type'] = skate_table.apply(lambda row: predictor_case(row, pred='forest_3', target='winning'), axis=1)
forest3_types = skate_table['forest_3_type'].value_counts()
print((accuracy(forest3_types), f1(forest3_types), informedness(forest3_types)))


(0.9306394501145595, 0.0, 0.0)


So we have some models that are giving us pretty good results for all of them. 

I should let you know that these models are only using a portion of the complete data to construct a model. To prevent overfitting, I will now test how good each of these models are by giving them data that the models have never predicted on. The model is going to test itself to see if it can correctly predict the times under 41 seconds, and return me the results. 

In [53]:
def oob_testing(forest, table, target):
    oobs = set()
    for i in range(len(forest)):
        temp = set(forest[i]['oob'])
        oobs = oobs.union(temp)
    left_out = table.loc[table.index.isin(oobs)]
    forest_oob = build_tree_iter(table, splitter_columns, target)
    return caser(left_out, forest_oob, target)

In [54]:
forest1_oob_types = oob_testing(forest1, skate_table, 'winning')
print(forest1_oob_types.sum())  # length of testing table
print((accuracy(forest1_oob_types), f1(forest1_oob_types), informedness(forest1_oob_types)))

4301
(0.9416414787258777, 0.38329238329238335, 0.2540007745868451)


In [55]:
forest2_oob_types = oob_testing(forest2, skate_table, 'winning')
print(forest2_oob_types.sum())  # length of testing table
print((accuracy(forest2_oob_types), f1(forest2_oob_types), informedness(forest2_oob_types)))

4776
(0.9413735343383585, 0.3805309734513274, 0.25194471536979335)


In [56]:
forest3_oob_types = oob_testing(forest3, skate_table, 'winning')
print(forest3_oob_types.sum())  # length of testing table
print((accuracy(forest3_oob_types), f1(forest3_oob_types), informedness(forest3_oob_types)))

4799
(0.9412377578662221, 0.3788546255506608, 0.25042126766264694)


Hm, so it appears that the testing is producing better results with less trees in the forest. It appears that the more trees we have the more difficult of a time they have making up their minds.

## Decision Trees

I will make a decision tree from the table by slicing the table into 10 parts and the builds the decision trees with 9/10th of the data by looking at each column in the splitter columns and determining which best predicts data. It then uses the remaining 1/10th of data to test against. After all that is done we can look at how these models performed.


In [57]:
max1_table = k_fold(skate_table, 10, 'winning', {'max-depth':1}, splitter_columns)
max1_table.describe()

,true_positive,false_positive,true_negative,false_negative,accuracy,f1,informedness
count,7.000000,9.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,11.714286,5.888889,441.500000,25.100000,0.936685,0.234883,0.153859
std,7.587584,3.822448,27.301811,16.515649,0.040028,0.186706,0.125712
min,1.000000,1.000000,401.000000,4.000000,0.879167,0.000000,-0.008677
25%,6.500000,4.000000,422.250000,11.250000,0.909420,0.026316,0.017420
50%,13.000000,6.000000,443.000000,24.500000,0.933333,0.306792,0.195833
75%,17.000000,7.000000,462.000000,37.750000,0.967708,0.399642,0.263825
max,21.000000,12.000000,475.000000,52.000000,0.989583,0.422222,0.293530


In [58]:
max2_table = k_fold(skate_table, 10, 'winning', {'max-depth':2}, splitter_columns)
max2_table.describe()

,true_positive,false_positive,true_negative,false_negative,accuracy,f1,informedness
count,7.000000,7.00000,10.000000,10.00000,10.000000,10.000000,10.000000
mean,10.000000,5.00000,443.300000,26.30000,0.937935,0.217308,0.137296
std,6.708204,3.05505,26.238437,17.93228,0.040644,0.171806,0.111724
min,1.000000,2.00000,401.000000,4.00000,0.879167,0.000000,-0.008677
25%,6.000000,3.00000,425.250000,11.25000,0.912545,0.029412,0.020062
50%,10.000000,5.00000,443.000000,24.50000,0.933333,0.271429,0.157229
75%,13.000000,5.50000,463.500000,39.25000,0.968750,0.368732,0.236699
max,21.000000,11.00000,476.000000,52.00000,0.991667,0.420000,0.272929


In [59]:
max3_table = k_fold(skate_table, 10, 'winning', {'max-depth':3}, splitter_columns)
max3_table.describe()

,true_positive,false_positive,true_negative,false_negative,accuracy,f1,informedness
count,6.000000,8.000000,10.000000,10.00000,10.000000,10.000000,10.000000
mean,11.500000,4.250000,443.400000,26.40000,0.937935,0.210088,0.134594
std,5.576737,1.488048,25.635045,18.04439,0.039647,0.186258,0.125524
min,5.000000,2.000000,401.000000,4.00000,0.879167,0.000000,-0.010846
25%,8.250000,3.500000,425.250000,11.50000,0.912545,0.000000,0.000000
50%,11.000000,5.000000,445.500000,25.50000,0.936458,0.279515,0.168214
75%,13.000000,5.000000,463.000000,39.25000,0.967187,0.376050,0.246449
max,21.000000,6.000000,476.000000,52.00000,0.991667,0.420000,0.272929


In [60]:
max4_table = k_fold(skate_table, 10, 'winning', {'max-depth':4}, splitter_columns)
max4_table.describe()

,true_positive,false_positive,true_negative,false_negative,accuracy,f1,informedness
count,7.000000,8.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,11.571429,6.125000,441.900000,25.200000,0.937309,0.245481,0.174548
std,6.996598,3.356763,26.010468,17.868657,0.039400,0.196750,0.160028
min,2.000000,2.000000,397.000000,4.000000,0.879167,0.000000,-0.008529
25%,9.000000,3.500000,425.250000,9.500000,0.911503,0.033333,0.021435
50%,10.000000,6.000000,442.500000,23.500000,0.934375,0.304010,0.184762
75%,13.000000,9.250000,461.750000,39.750000,0.966667,0.393900,0.259822
max,25.000000,10.000000,476.000000,50.000000,0.991667,0.486486,0.478355


In [61]:
max5_table = k_fold(skate_table, 10, 'winning', {'max-depth':5}, splitter_columns)
max5_table.describe()

,true_positive,false_positive,true_negative,false_negative,accuracy,f1,informedness
count,7.000000,9.000000,10.000000,10.000000,10.000000,10.000000,10.000000
mean,11.000000,5.555556,441.800000,25.600000,0.936268,0.232116,0.152134
std,7.393691,2.962731,25.624641,17.506824,0.038205,0.183175,0.127318
min,2.000000,2.000000,397.000000,4.000000,0.879167,0.000000,-0.010661
25%,7.000000,4.000000,425.250000,11.500000,0.911503,0.032258,0.020893
50%,10.000000,5.000000,442.000000,23.500000,0.933333,0.304010,0.184762
75%,13.000000,7.000000,462.250000,39.750000,0.965625,0.357942,0.258739
max,25.000000,10.000000,476.000000,50.000000,0.991667,0.462963,0.317896


Here are my results placed into a chart to help see which is the best depth.

Hm, interesting. We see that the models go down in quality and then boost up at depth 4 only to depreciate again. Overall these decision trees are not preforming well compared to our random forest model.

## Conclusions

So, we have tested a random forest model and a decision tree that has a cross validation using k-fold. Our results are that a random tree with 5 trees produces the best results and a decision tree of depth 4 produces the best.

Looking at the two we can see that the random forest of size 5 beats out the decision tree of depth 4 on every metric. So we can confidently proceed forward and try and find tune our results.

In [62]:
import os

home_path =  os.path.expanduser('~')

file_path = '\Documents\CIS 399\ '

file_name = 'final_project_wrangled.csv'

skate_table.to_csv(home_path + file_path + file_name, index=False)